In [1]:
from emrgpt.model import TimelineBasedEmrGPT
import torch

*** EMR GPT ***


In [2]:
model = TimelineBasedEmrGPT(
    n_event_types=13,
    d_model=384,
    block_size=24,
    max_len=24,
    n_head=6,
    n_layer=6,
    dropout=0.2,
).to('cuda')

model.load_state_dict(torch.load('cache/savedmodels/TimelineBasedEmrGPT.pt'))
model.eval()
model

TimelineBasedEmrGPT(
  (proj): Linear(in_features=13, out_features=384, bias=True)
  (positional_encoding): FixedPositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (blocks): Sequential(
    (0): AkDecoderBlock(
      (self_attention): AkMultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x AkSelfAttentionHead(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (feedforward): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          

In [3]:
g = model.generate()
g

tensor([[  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
           0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
        [115.0488, 142.8280,  73.8695,  19.7809,  12.1723, 128.4802, 146.7883,
           9.2757,  -9.4398,   8.6896,   5.4671,   7.9984,  10.0389],
        [115.5839, 143.2981,  74.4189,  19.9794,  12.5017, 128.9812, 147.2423,
           9.2665,  -9.4163,   8.7174,   5.4770,   8.0228,  10.0824],
        [115.5863, 143.3001,  74.4213,  19.9803,  12.5031, 128.9833, 147.2442,
           9.2665,  -9.4162,   8.7175,   5.4771,   8.0229,  10.0826],
        [115.5863, 143.3001,  74.4214,  19.9803,  12.5032, 128.9834, 147.2442,
           9.2665,  -9.4162,   8.7175,   5.4771,   8.0229,  10.0827],
        [115.5864, 143.3002,  74.4214,  19.9803,  12.5032, 128.9835, 147.2443,
           9.2665,  -9.4162,   8.7175,   5.4771,   8.0229,  10.0827],
        [115.5864, 143.3002,  74.4215,  19.9803,  12.5032, 128.9835, 147.2443,
           9.2665,  -9.4162

In [11]:
g[:,8:13]

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-9.4398,  8.6896,  5.4671,  7.9984, 10.0389],
        [-9.4163,  8.7174,  5.4770,  8.0228, 10.0824],
        [-9.4162,  8.7175,  5.4771,  8.0229, 10.0826],
        [-9.4162,  8.7175,  5.4771,  8.0229, 10.0827],
        [-9.4162,  8.7175,  5.4771,  8.0229, 10.0827],
        [-9.4162,  8.7175,  5.4771,  8.0229, 10.0827],
        [-9.4162,  8.7175,  5.4771,  8.0229, 10.0827],
        [-9.4162,  8.7175,  5.4771,  8.0229, 10.0827],
        [-9.4162,  8.7175,  5.4771,  8.0229, 10.0827],
        [-9.4162,  8.7175,  5.4771,  8.0229, 10.0827],
        [-9.4162,  8.7175,  5.4771,  8.0229, 10.0827],
        [-9.4162,  8.7175,  5.4771,  8.0229, 10.0827]], device='cuda:0',
       grad_fn=<SliceBackward0>)